In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import stats

In [ ]:
def normalization(scaler_, df_):
    data_array = df_.values
    data_array = data_array.astype(float)
    
    data_array_normalized = scaler_.transform(data_array)
    df_normalized = pd.DataFrame(data_array_normalized, columns = list(df_.columns))
    
    return df_normalized

In [ ]:
def create_sequences(df, window_size, n_overlap):
    n_rows = len(df)
    if n_overlap == 0:
        n_chunks = n_rows // window_size
        return np.array([df.iloc[i * window_size:(i + 1) * window_size].values for i in range(n_chunks)])
    else:
        overlap = window_size - n_overlap 
        sequences = []
        for start in range(0, n_rows - window_size + 1, overlap):
            end = start + window_size
            sequence = df.iloc[start:end].values
            sequences.append(sequence)
        return np.array(sequences)

In [ ]:
def calc_kurtosis(x):
    value = stats.kurtosis(x, fisher = True, bias = False) 
    if pd.isna(value):
        return 0.0
    else:
        return value

In [ ]:
def calc_skewness(x):
    value = stats.skew(x, bias = False) 
    if pd.isna(value):
        return 0.0
    else:
        return value

In [ ]:
def custom_mape(orig, reconstr):
    # Compute MAPE trying to avoid division by 0 (or too small factors)
    y_true, y_pred = np.array(orig), np.array(reconstr)
    y_true_adjusted = np.where(abs(y_true) <= 0.0001, y_true + 0.01, y_true)
    y_pred_adjusted = np.where(abs(y_true) <= 0.0001, y_pred + 0.01, y_pred)
    return np.mean(np.abs((y_true_adjusted - y_pred_adjusted) / y_true_adjusted))

In [ ]:
def compute_failed_requests(rides_not_served, rides_requested, a, b, c):
    failed = rides_not_served / rides_requested
    weight = min(1, a*np.log(rides_requested+b)+c)
    w_failed = failed * weight
    return w_failed

In [ ]:
def compute_thresholds(failed_req):
    return np.percentile(failed_req, 95), np.percentile(failed_req, 97), np.percentile(failed_req, 99)

In [ ]:
def compute_time_above(hardiness, t):    
    timestamps = 0
    max_timestamps = 0
    for i in range(t, len(hardiness)):
        if hardiness[i-60:i].mean() >= hardiness[i-t-60:i-60].mean():
            timestamps += 1
        else:
            max_timestamps = max(timestamps, max_timestamps)
            timestamps = 0
    return max_timestamps

In [ ]:
def compute_resilience(hardiness, threshold_r, t):
    timestamps = 0
    max_timestamps = 0
    exceeding_index = 0
    found = False
    for i in range(t, len(hardiness)):
        if hardiness[i-60:i].mean() >= hardiness[i-t-60:i-60].mean():
            timestamps += 1
            if timestamps > threshold_r and not found:
                if exceeding_index == 0:
                    exceeding_index = i
                    found = True
        else:
            max_timestamps = max(timestamps, max_timestamps)
            timestamps = 0
    if max_timestamps > threshold_r:
        return exceeding_index, max_timestamps
    return 0, max_timestamps

In [ ]:
def compute_failure(mean_error, threshold, detect_time, confidence):
    for start in range(0, len(mean_error)):
        if start + detect_time < len(mean_error):
            window = mean_error[start:start + detect_time]
        else:
            window = mean_error[start:]
        timestamps_above = np.sum(window > threshold)
        if timestamps_above / detect_time >= confidence:
            i = window.index[0]
            while i < window.index[len(window) - 1]:
                if window[i] > threshold:
                    return i + detect_time
                else:
                    i += 1

In [1]:
def exceed_thresholds(index, threshold_dict, perc_95, perc_97, perc_99):
    found_95 = False
    found_97 = False
    found_99 = False
    for i in range(0, len(index)):
        if index[i] >= perc_95 and not found_95:
            threshold_dict['perc_95'] = i
            found_95 = True
        if index[i] >= perc_97 and not found_97:
            threshold_dict['perc_97'] = i
            found_97 = True
        if index[i] >= perc_99 and not found_99:
            threshold_dict['perc_99'] = i
            found_99 = True
    return threshold_dict